# Azure AI agentai su Model Context Protocol (MCP) palaikymu - Python

Šiame užrašų knygelėje demonstruojama, kaip naudoti Azure AI agentus su Model Context Protocol (MCP) įrankiais Python kalba. Parodoma, kaip sukurti išmanų agentą, kuris gali pasinaudoti išoriniais MCP serveriais (pvz., Microsoft Learn) siekiant išplėsti galimybes naudojant autentifikaciją be raktų.


## Įdiekite reikalingus Python paketus

Pirmiausia reikia įdiegti būtinus Python paketus:
- **azure-ai-projects**: Pagrindinis Azure AI Projects SDK
- **azure-ai-agents**: Azure AI Agents SDK, skirtas agentų kūrimui ir valdymui
- **azure-identity**: Užtikrina autentifikaciją be raktų naudojant DefaultAzureCredential
- **mcp**: Model Context Protocol įgyvendinimas Python


## Privalumai naudojant autentifikaciją be raktų

Šiame užrašų knygelėje demonstruojama **autentifikacija be raktų**, kuri suteikia keletą privalumų:
- ✅ **Nereikia valdyti API raktų** - Naudojama Azure tapatybės pagrindu veikianti autentifikacija
- ✅ **Padidintas saugumas** - Jokie slapti duomenys nėra saugomi kode ar konfigūracijos failuose
- ✅ **Automatinis kredencialų atnaujinimas** - Azure tvarko kredencialų gyvavimo ciklą
- ✅ **Prieiga pagal vaidmenis** - Naudojama Azure RBAC smulkiam leidimų valdymui
- ✅ **Daugiaplinkės aplinkos palaikymas** - Veikia sklandžiai tiek kūrimo, tiek gamybos aplinkose

`DefaultAzureCredential` automatiškai pasirenka geriausią prieinamą kredencialų šaltinį:
1. **Valdomoji tapatybė** (kai veikia Azure aplinkoje)
2. **Azure CLI** kredencialai (vietinio kūrimo metu)
3. **Visual Studio** kredencialai
4. **Aplinkos kintamieji** (jei sukonfigūruoti)
5. **Interaktyvi naršyklės autentifikacija** (kaip atsarginis variantas)


## Nustatymas be raktų autentifikacijos

**Reikalavimai autentifikacijai be raktų:**

### Vietiniam vystymui:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure aplinkoms:
- Įgalinkite **sistemos priskirtą valdomą tapatybę** savo Azure resurse
- Priskirkite tinkamus **RBAC vaidmenis** valdomai tapatybei:
  - `Cognitive Services OpenAI User` prieigai prie Azure OpenAI
  - `AI Developer` prieigai prie Azure AI projektų

### Aplinkos kintamieji (Pasirinktinai):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Jokių API raktų ar prisijungimo eilučių nereikia!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importuokite reikalingas bibliotekas

Importuokite būtinas Python modulius:  
- **os, time**: Standartinės Python bibliotekos aplinkos kintamiesiems ir uždelsimams  
- **AIProjectClient**: Pagrindinis klientas Azure AI projektams  
- **DefaultAzureCredential**: Autentifikacija be raktų Azure paslaugoms  
- **MCP susijusios klasės**: MCP įrankių kūrimui ir valdymui bei patvirtinimų tvarkymui  


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigūruokite MCP serverio nustatymus

Nustatykite MCP serverio konfigūraciją naudodami aplinkos kintamuosius su numatytaisiais parametrais:
- **MCP_SERVER_URL**: MCP serverio URL (numatytasis – Microsoft Learn API)
- **MCP_SERVER_LABEL**: Etiketė MCP serveriui identifikuoti (numatytoji – "mslearn")

Šis metodas leidžia lanksčiai konfigūruoti skirtingose aplinkose.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Sukurkite Azure AI Projekto Klientą (Autentifikacija be raktų)

Inicializuokite Azure AI Projekto klientą naudodami **autentifikaciją be raktų**:
- **endpoint**: Azure AI Foundry projekto galinio taško URL
- **credential**: Naudoja `DefaultAzureCredential()` saugiai, be raktų autentifikacijai
- **Nereikia API raktų**: Automatiškai aptinka ir naudoja geriausią prieinamą kredencialą

**Autentifikacijos eiga:**
1. Tikrina valdomą tapatybę (Azure aplinkose)
2. Pereina prie Azure CLI kredencialų (vietinei plėtrai)
3. Naudoja kitus prieinamus kredencialų šaltinius, jei reikia

Šis metodas pašalina poreikį valdyti API raktus ar prisijungimo eilutes jūsų kode.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Sukurti MCP Įrankio Apibrėžimą

Sukurkite MCP įrankį, kuris jungiasi prie Microsoft Learn MCP serverio:
- **server_label**: MCP serverio identifikatorius
- **server_url**: MCP serverio URL adresas
- **allowed_tools**: Pasirinktinai pateikiamas sąrašas, apribojantis, kokie įrankiai gali būti naudojami (tuščias sąrašas leidžia naudoti visus įrankius)

Šis įrankis leis agentui pasiekti Microsoft Learn dokumentaciją ir išteklius.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Sukurkite agentą ir vykdykite pokalbį (darbas be raktų)

Ši išsami dalis parodo visą **darbo be raktų agento procesą**:

1. **Sukurkite AI agentą**: Sukonfigūruokite agentą su GPT-4.1 nano modeliu ir MCP įrankiais  
2. **Sukurkite giją**: Sukurkite pokalbio giją bendravimui  
3. **Siųskite žinutę**: Paklauskite agento apie Azure OpenAI ir OpenAI skirtumus  
4. **Tvarkykite įrankių patvirtinimus**: Automatiškai patvirtinkite MCP įrankių naudojimą, kai to reikia  
5. **Stebėkite vykdymą**: Sekite agento progresą ir atlikite reikiamus veiksmus  
6. **Rodykite rezultatus**: Parodykite pokalbio ir įrankių naudojimo detales  

**Darbo be raktų ypatybės:**
- ✅ **Nėra įrašytų slaptažodžių** – Visa autentifikacija vykdoma naudojant Azure identitetą  
- ✅ **Saugumas pagal nutylėjimą** – Naudojama vaidmenimis pagrįsta prieigos kontrolė  
- ✅ **Supaprastintas diegimas** – Nereikia valdyti prisijungimo duomenų  
- ✅ **Tinka auditui** – Visa prieiga sekama per Azure identitetą  

Agentas naudos MCP įrankius, kad pasiektų Microsoft Learn išteklius, užtikrindamas visišką saugumą ir nereikalaudamas API raktų valdymo.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
